In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import xml.etree.ElementTree as et 
import os, glob

In [2]:
# file to a variable under the name
# data
with open('q1eta1.citations.xml', 'r') as f:
    data = f.read()

# the beautifulsoup parser, storing
# the returned object
Bs_data = BeautifulSoup(data, "xml")
print (Bs_data)

<?xml version="1.0" encoding="utf-8"?>
<nite:root id="q1eta1.citations" xmlns:nite="http://nite.sourceforge.net/">
<citation id="q1eta1.cit.1" repetition="1">
<nite:pointer href="maptask-landmarks.xml#id(m3_wheat_fields)" role="landmark"/>
<nite:child href="q1eta1.wordlist.xml#id(q1eta1.1)"/>
<nite:child href="q1eta1.wordlist.xml#id(q1eta1.2)"/>
</citation>
<citation id="q1eta1.cit.6" repetition="1">
<nite:pointer href="maptask-landmarks.xml#id(m9_stony_desert)" role="landmark"/>
<nite:child href="q1eta1.wordlist.xml#id(q1eta1.6)"/>
<nite:child href="q1eta1.wordlist.xml#id(q1eta1.7)"/>
</citation>
<citation id="q1eta1.cit.11" repetition="1">
<nite:pointer href="maptask-landmarks.xml#id(m6_graveyard)" role="landmark"/>
<nite:child href="q1eta1.wordlist.xml#id(q1eta1.11)"/>
</citation>
<citation id="q1eta1.cit.15" repetition="1">
<nite:pointer href="maptask-landmarks.xml#id(m6_buffalo)" role="landmark"/>
<nite:child href="q1eta1.wordlist.xml#id(q1eta1.15)"/>
</citation>
<citation id="q1e

In [3]:
def parse_XML(xml_file, df_cols): 
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)      
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

In [7]:
path = os.getcwd()
xml = glob.glob(os.path.join(path, "*.xml"))
trial = pd.DataFrame()
trial1 = pd.DataFrame()
t = pd.DataFrame()
for f in xml:
    file = parse_XML(f, ['id'])
    trial = trial.append(file)
    file1 = parse_XML(f, ['repetition'])
    trial1 = trial1.append(file1)
    one = pd.concat([file, trial], axis='rows')
    two = pd.concat([file1, trial1], axis='rows')
    root = et.parse(f).getroot()
    for child in root:
        for grandchild in child:           
            t = t.append(grandchild.attrib, ignore_index=True)
one = one.reset_index(drop=True)
two = two.reset_index(drop=True)
full = pd.concat([one, two, t], axis = 1)
full

,id,repetition,href,role
0,q8nta2.cit.1,1,maptask-landmarks.xml#id(m3_wheat_fields),landmark
1,q8nta2.cit.6,1,q1nta2.wordlist.xml#id(q1nta2.1),NaN
2,q8nta2.cit.11,1,q1nta2.wordlist.xml#id(q1nta2.2),NaN
3,q8nta2.cit.16,1,maptask-landmarks.xml#id(m9_stony_desert),landmark
4,q8nta2.cit.21,1,q1nta2.wordlist.xml#id(q1nta2.5),NaN
...,...,...,...,...
21075,NaN,NaN,q8nta2.wordlist.xml#id(q8nta2.510),NaN
21076,NaN,NaN,q8nta2.wordlist.xml#id(q8nta2.511),NaN
21077,NaN,NaN,maptask-landmarks.xml#id(m13_collapsed_shelter),landmark
21078,NaN,NaN,q8nta2.wordlist.xml#id(q8nta2.515),NaN


In [5]:
parse_XML('q1eta1.citations.xml', ['id'])

,id
0,q1eta1.cit.1
1,q1eta1.cit.6
2,q1eta1.cit.11
3,q1eta1.cit.15
4,q1eta1.cit.19
...,...
115,q1eta1.cit.575
116,q1eta1.cit.580
117,q1eta1.cit.585
118,q1eta1.cit.590


In [8]:
full.to_csv('citations.csv', sep=',', encoding='utf-8')